## About WIS
Good ressources:
-  Supplement of Cramer et al.
- code cramer et al. here https://github.com/reichlab/covid19-forecast-evals
- obviously https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008618#sec015
- [git clone https://github.com/adrian-lison/interval-scoring.git](https://github.com/adrian-lison/interval-scoring/tree/master) Adrian Lison's code for WIS
- https://epiforecasts.io/scoringutils/ Scoring utils package -- perhaps best to use ?

In [60]:
from interval_scoring import scoring
import pandas as pd
import numpy as np
from tqdm import tqdm

In [42]:
f

NameError: name 'f' is not defined

In [54]:
def score_forecasts(gt, forecasts):
    if isinstance(gt, str):
        gt = pd.read_csv(gt)
    if isinstance(forecasts, str):
        forecast = pd.read_csv(forecast)
    gt = gt[gt["location"].isin(forecasts["location"])]
    
    alphas= np.array(sorted(forecasts["quantile"].unique()))[:11]*2
    
        
    gt_piv = gt[gt["date"].isin(forecasts.target_end_date)].pivot(index="date", columns="location", values="value").sort_index()
    
    
    wis_total = pd.DataFrame(0, index=gt_piv.index, columns=gt_piv.columns)
    wis_sharpness = pd.DataFrame(0, index=gt_piv.index, columns=gt_piv.columns)
    wis_calibration = pd.DataFrame(0, index=gt_piv.index, columns=gt_piv.columns)
    
    for target in gt_piv.index:
        f = forecasts[forecasts["target_end_date"] == target]
        q_dict = {}
        for q in f["quantile"].unique():
            q_dict[float(q)] = f[f["quantile"]==q].pivot(index="target_end_date", columns="location", values="value").sort_index().to_numpy().ravel()
        wis_total.loc[target], wis_sharpness.loc[target], wis_calibration.loc[target] =  scoring.weighted_interval_score(observations=gt_piv.loc[target].to_numpy(), 
                                                                            alphas=alphas, 
                                                                            q_dict=q_dict, 
                                                                            weights=alphas/2)
    
        wis_total.loc[target]
    
    return wis_total, wis_sharpness, wis_calibration

In [64]:
fdates = pd.date_range("2022-11-14", "2023-05-15", freq="W-MON")

gt = pd.read_csv("Flusight/Flusight-forecast-data/data-truth/truth-Incident Hospitalizations.csv")

scores = {}
for date in tqdm(fdates):
    date = date.date()
    try:
        forecasts = pd.read_csv(f"Flusight/Flusight-forecast-data/data-forecasts/UNC_IDD-InfluPaint/{str(date)}-UNC_IDD-InfluPaint.csv")
        this_date=True
    except FileNotFoundError:
        print(f"skipping {date}")
        this_date=False
    if this_date:
        wis_total, wis_sharpness, wis_calibration = score_forecasts(gt, forecasts)
        scores[date] = wis_total
        

  7%|▋         | 2/27 [00:00<00:07,  3.30it/s]

skipping 2022-11-28


 41%|████      | 11/27 [00:02<00:03,  4.11it/s]

skipping 2023-01-30


100%|██████████| 27/27 [00:06<00:00,  4.22it/s]


ValueError: If using all scalar values, you must pass an index

In [95]:
q_dict[0.01]

location,01,02,04,05,06,08,09,10,11,12,...,47,48,49,50,51,53,54,55,56,US
target_end_date,,,,,,,,,,,,,,,,,,,,,
2022-11-19,215.826505,8.474259,74.415518,183.956470,784.833063,47.760973,55.607553,8.336416,20.551658,473.108468,...,318.003985,880.318480,10.308967,4.014218,380.289382,54.964502,83.879923,49.470893,4.403230,8101.194765
2022-11-26,175.979997,7.861236,69.607775,175.703556,851.237740,50.555501,58.522854,9.755225,22.392716,508.415168,...,329.555027,682.858954,10.554381,5.190021,431.405683,55.947064,87.191237,61.584991,4.186020,8454.814988
2022-12-03,134.136669,7.410911,58.545103,150.357802,827.308120,49.028275,53.881750,10.235729,22.258382,499.325026,...,309.782306,498.662739,8.740848,6.199420,442.539245,55.935754,83.717269,69.828478,3.936483,8132.113769
2022-12-10,101.101871,6.004188,47.394595,117.538420,733.719733,43.398954,47.046461,9.786414,19.535840,453.920150,...,261.727654,338.350211,7.970135,5.699878,414.857555,51.242794,75.599867,68.511714,3.346635,7181.866700


In [65]:
scoring.weighted_interval_score?

Signature:
scoring.weighted_interval_score(
    observations,
    alphas,
    q_dict,
    weights=None,
    percent=False,
    check_consistency=True,
)
Docstring:
Compute weighted interval scores for an array of observations and a number of different predicted intervals.

This function implements the WIS-score (2). A dictionary with the respective (alpha/2)
and (1-(alpha/2)) quantiles for all alpha levels given in `alphas` needs to be specified.

Parameters
----------
observations : array_like
    Ground truth observations.
alphas : iterable
    Alpha levels for (1-alpha) intervals.
q_dict : dict
    Dictionary with predicted quantiles for all instances in `observations`.
weights : iterable, optional
    Corresponding weights for each interval. If `None`, `weights` is set to `alphas`, yielding the WIS^alpha-score.
percent: bool, optional
    If `True`, score is scaled by absolute value of observations to yield the double absolute percentage error. Default is `False`.
check_consistency